# Simulating Deuteron $N=2$ Hamiltonian with XACC-VQE

In [ ]:
from  openfermionpsi4  import run_psi4
from openfermion.hamiltonians import MolecularData
import numpy as np
import xacc
from xacc import InstructionParameter
import xaccvqe as vqe
from xaccvqe import PauliOperator
%matplotlib inline

## Import necessary libraries, create the Hamiltonian and Ansatz Circuits

In [ ]:
xacc.Initialize(['--compiler','quil'])

# Specify the Accelerator to run on, 
# default to TNQVM MPS simulator
qpuName = 'tnqvm'
qpu = xacc.getAccelerator(qpuName)

# Construct the H2 Hamiltonian (see Eq 4 of paper)
H2 = PauliOperator(5.906709445) + \
    PauliOperator({0:'X',1:'X'}, -2.1433) + \
    PauliOperator({0:'Y',1:'Y'}, -2.1433) + \
    PauliOperator({0:'Z'}, .21829) + \
    PauliOperator({1:'Z'}, -6.125)

# Code up and compile the H2 ansatz in Quil
srcH2 = """__qpu__ ansatz(AcceleratorBuffer, double t0) {
X 0
RY(t0) 1
CNOT 1 0
}"""
ansatzH2 = xacc.compileKernel(qpu, srcH2) 


## Parameter Sweep

In [ ]:
# Sweep over the given angle range for H2 and plot
angle_range = np.linspace(-np.pi,np.pi,50)
energiesH2 = [vqe.execute(H2, **{'task':'compute-energy', 
                                        'vqe-params':str(t), 
                                        'ansatz':ansatzH2}).energy
            for t in angle_range]
import matplotlib.pyplot as plt
plt.figure(figsize=(9,5))
plt.xlabel(r'$\theta$')
plt.ylabel(r'$\langle H\rangle$')
plt.plot(angle_range, energiesH2, '-', color='k', label=r'tnqvm')
plt.legend()
plt.show()


## VQE routine

In [ ]:
# Execute H2 VQE routine
vqeResultH2 = vqe.execute(H2, **{'task':'vqe', 'ansatz':ansatzH2})
print('H2 (Optimal Angle, Energy) = (', vqeResultH2.angles, ',', vqeResultH2.energy, ')')

## Exercise - Run the Parameter Sweep on IBM simulator